In [1]:
import os
#os.environ["NUMBA_LOOP_VECTORIZE"] = "0"
#os.environ["NUMBA_OPT"] = "0"
#os.environ["NUMBA_DUMP_ASSEMBLY"] = "1"
from llvmlite import binding as llvm
#llvm.set_option("", "--debug-only=loop-vectorize")

from numba import njit
import numpy as np

In [2]:
DATA = np.linspace(1, 1_000_000, 1_000_000, dtype=np.uint64)

@njit
def sum(arr):
    total = 0
    for i in range(len(arr)):
        total += arr[i]
    return total

sum(DATA)

500000500000

In [3]:
import sys
sys.path.append("../src")
%load_ext book_magics

In [4]:
%%compare_timing
sum(DATA)
np.sum(DATA)

|     Code     |Time to run (microseconds)|
|--------------|-------------------------:|
|`sum(DATA)`   |                   138.206|
|`np.sum(DATA)`|                   161.899|


In [3]:
%timeit sum(DATA)

5.02 ms ± 29.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
    @njit
    def generate_random_numbers(n):
        result = np.empty((n,), dtype=np.uint64)
        random_number = 1
        for i in range(n):
            random_number = (random_number * 437799614237992725) % (2 ** 61 - 1)
            result[i] = random_number
        return result
    
    generate_random_numbers(3)

array([ 437799614237992725, 1692544495229593526,  262370722887384048],
      dtype=uint64)

In [5]:
%timeit generate_random_numbers(1_000_000)

11.8 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
@njit
def generate_random_numbers_2(n):
    result = np.empty((n,), dtype=np.uint64)
    for i in range(n):
        random_number = (i * 437799614237992725) % (2 ** 61 - 1)
        result[i] = random_number
    return result

generate_random_numbers_2(30)

-------------------------------OPTIMIZED DUMP nrt-------------------------------
; ModuleID = 'nrt'
source_filename = "<string>"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@.const.pickledata.140635430347264 = internal constant [102 x i8] c"\80\04\95[\00\00\00\00\00\00\00\8C\08builtins\94\8C\0CRuntimeError\94\93\94\8C6numba jitted function aborted due to unresolved symbol\94\85\94N\87\94."
@.const.pickledata.140635430347264.sha1 = internal constant [20 x i8] c"\97\BE\DC\DF\EC\8E\80\B7\09>P\CE%\EDV\F7r\0E\0C\9C"
@.const.picklebuf.140635430347264 = internal constant { i8*, i32, i8*, i8*, i32 } { i8* getelementptr inbounds ([102 x i8], [102 x i8]* @.const.pickledata.140635430347264, i32 0, i32 0), i32 102, i8* getelementptr inbounds ([20 x i8], [20 x i8]* @.const.pickledata.140635430347264.sha1, i32 0, i32 0), i8* null, i32 0 }

; Function Attrs: mustprogress nofree norecurse nounwind willreturn
d


LV: Checking a loop in "_ZN8__main__25generate_random_numbers_2B2v1B38c8tJTIcFKzyF2ILShI4CrgQElQb6HczSBAA_3dEx" from generate_random_numbers_2
LV: Loop hints: force=? width=0 interleave=0
LV: Not vectorizing: No #pragma vectorize enable.
LV: Loop hints prevent vectorization.


array([                  0,  437799614237992725,  875599228475985450,
       1313398842713978175, 1751198456951970900, 2188998071189963625,
        320954676214262399,  758754290452255124, 1196553904690247849,
       1634353518928240574, 2072153133166233299,  204109738190532073,
        641909352428524798, 1079708966666517523, 1517508580904510248,
       1955308195142502973,   87264800166801747,  525064414404794472,
        962864028642787197, 1400663642880779922, 1838463257118772647,
       2276262871356765372,  408219476381064146,  846019090619056871,
       1283818704857049596, 1721618319095042321, 2159417933333035046,
        291374538357333820,  729174152595326545, 1166973766833319270],
      dtype=uint64)

In [7]:
%timeit generate_random_numbers_2(1_000_000)

6.99 ms ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
@njit
def generate_random_numbers_3(n):
    result = np.empty((n,), dtype=np.uint64)
    for i in range(n // 2):
        random_number1 = ((i * 2) * 437799614237992725) % (2 ** 61 - 1)
        result[i * 2] = random_number1
        random_number2 = ((i * 2 + 1) * 437799614237992725) % (2 ** 61 - 1)
        result[i * 2 + 1] = random_number2
    result[-1] = ((n - 1) * 437799614237992725) % (2 ** 61 - 1)
    return result

generate_random_numbers_3(30)

array([                  0,  437799614237992725,  875599228475985450,
       1313398842713978175, 1751198456951970900, 2188998071189963625,
        320954676214262399,  758754290452255124, 1196553904690247849,
       1634353518928240574, 2072153133166233299,  204109738190532073,
        641909352428524798, 1079708966666517523, 1517508580904510248,
       1955308195142502973,   87264800166801747,  525064414404794472,
        962864028642787197, 1400663642880779922, 1838463257118772647,
       2276262871356765372,  408219476381064138,  846019090619056863,
       1283818704857049588, 1721618319095042313, 2159417933333035038,
        291374538357333812,  729174152595326537, 1166973766833319262],
      dtype=uint64)

In [9]:
%timeit generate_random_numbers_3(1_000_000)

5.16 ms ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
from importlib import reload
import sys
os.environ["NUMBA_OPT"] = "3"
to_remove = [m for m in sys.modules if m.startswith("numba")]
for m in to_remove:
    sys.modules.pop(m)
import numba
reload(numba)
from numba import njit

In [11]:
@njit
def generate_random_numbers_2b(n):
    result = np.empty((n,), dtype=np.uint64)
    for i in range(n):
        random_number = (i * 437799614237992725) % (2 ** 61 - 1)
        result[i] = random_number
    return result

generate_random_numbers_2b(30)


LV: Checking a loop in "_ZN8__main__26generate_random_numbers_2bB2v1B38c8tJTIcFKzyF2ILShI4CrgQElQb6HczSBAA_3dEx" from generate_random_numbers_2b
LV: Loop hints: force=? width=0 interleave=0
LV: Not vectorizing: No #pragma vectorize enable.
LV: Loop hints prevent vectorization.


array([                  0,  437799614237992725,  875599228475985450,
       1313398842713978175, 1751198456951970900, 2188998071189963625,
        320954676214262399,  758754290452255124, 1196553904690247849,
       1634353518928240574, 2072153133166233299,  204109738190532073,
        641909352428524798, 1079708966666517523, 1517508580904510248,
       1955308195142502973,   87264800166801747,  525064414404794472,
        962864028642787197, 1400663642880779922, 1838463257118772647,
       2276262871356765372,  408219476381064146,  846019090619056871,
       1283818704857049596, 1721618319095042321, 2159417933333035046,
        291374538357333820,  729174152595326545, 1166973766833319270],
      dtype=uint64)

In [12]:
%timeit generate_random_numbers_2b(1_000_000)

497 µs ± 4.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:
# Increases linearly from 1 to 1,000,000:
PREDICTABLE_DATA = np.linspace(1, 1_000_000, 1_000_000, dtype=np.uint64)
# Shuffled randomly:
RANDOM_DATA = PREDICTABLE_DATA.copy()
np.random.shuffle(RANDOM_DATA)

In [42]:
from numba import float64

@njit 
def count_increasing_decreasing(arr):
    previous = 0
    increasing = 0
    unchanged = 0
    for value in arr:
        if value == previous:
            unchanged += 1
        elif value > previous:
            increasing += 1
        previous = value
    decreasing = len(arr) - increasing - unchanged
    return increasing, unchanged, decreasing

count_increasing_decreasing([1, 2, 3, 2, 2, 4])

/home/itamarst/devel/business/performance/low-level-performance/venv/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'arr' of function 'count_increasing_decreasing'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../../tmp/ipykernel_3649244/89310949.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))

LV: Checking a loop in "_ZN8__main__27count_increasing_decreasingB3v15B38c8tJTIcFKzyF2ILShI4CrgQElQb6HczSBAA_3dE42reflected_20list_28int64_29_3civ_3dNone_3e" from count_increasing_decreasing
LV: Loop hints: force=? width=0 interleave=0
LV: Not vectorizing: No #pragma vectorize enable.
LV: Loop hints prevent vectorization.

LV: Checking a loop in "_ZN7cpython8__main__2

(4, 1, 1)

In [43]:
%timeit count_increasing_decreasing(PREDICTABLE_DATA)
%timeit count_increasing_decreasing(RANDOM_DATA)


LV: Checking a loop in "_ZN8__main__27count_increasing_decreasingB3v16B38c8tJTIcFKzyF2ILShI4CrgQElQb6HczSBAA_3dE5ArrayIyLi1E1C7mutable7alignedE" from count_increasing_decreasing
LV: Loop hints: force=? width=0 interleave=0
LV: Not vectorizing: No #pragma vectorize enable.
LV: Loop hints prevent vectorization.


498 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
3.25 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
from numba import float64

@njit 
def count_increasing_decreasing_avoid_conditionals(arr):
    previous = 0
    increasing = 0
    unchanged = 0
    for value in arr:
        unchanged += 1 if value == previous else 0
        increasing += 1 if value > previous else 0
        previous = value
    decreasing = len(arr) - increasing - unchanged
    return increasing, unchanged, decreasing

count_increasing_decreasing_avoid_conditionals([1, 2, 3, 2, 2, 4])

----------------------------------ASSEMBLY nrt----------------------------------
	.text
	.file	"<string>"
	.globl	nrt_atomic_add
	.p2align	4, 0x90
	.type	nrt_atomic_add,@function
nrt_atomic_add:
	movl	$1, %eax
	lock		xaddq	%rax, (%rdi)
	incq	%rax
	retq
.Lfunc_end0:
	.size	nrt_atomic_add, .Lfunc_end0-nrt_atomic_add

	.globl	nrt_atomic_sub
	.p2align	4, 0x90
	.type	nrt_atomic_sub,@function
nrt_atomic_sub:
	movq	$-1, %rax
	lock		xaddq	%rax, (%rdi)
	decq	%rax
	retq
.Lfunc_end1:
	.size	nrt_atomic_sub, .Lfunc_end1-nrt_atomic_sub

	.globl	nrt_atomic_cas
	.p2align	4, 0x90
	.type	nrt_atomic_cas,@function
nrt_atomic_cas:
	movq	%rsi, %rax
	xorl	%esi, %esi
	lock		cmpxchgq	%rdx, (%rdi)
	sete	%sil
	movq	%rax, (%rcx)
	movl	%esi, %eax
	retq
.Lfunc_end2:
	.size	nrt_atomic_cas, .Lfunc_end2-nrt_atomic_cas

	.globl	NRT_MemInfo_data_fast
	.p2align	4, 0x90
	.type	NRT_MemInfo_data_fast,@function
NRT_MemInfo_data_fast:
	movq	24(%rdi), %rax
	retq
.Lfunc_end3:
	.size	NRT_MemInfo_data_fast, .Lfunc_end3-NRT_MemInf

/home/itamarst/devel/business/performance/low-level-performance/venv/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'arr' of function 'count_increasing_decreasing_avoid_conditionals'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../../tmp/ipykernel_3673217/2673960763.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))

LV: Checking a loop in "_ZN8__main__46count_increasing_decreasing_avoid_conditionalsB2v1B38c8tJTIcFKzyF2ILShI4CrgQElQb6HczSBAA_3dE42reflected_20list_28int64_29_3civ_3dNone_3e" from count_increasing_decreasing_avoid_conditionals
LV: Loop hints: force=? width=0 interleave=0
LV: Not vectorizing: No #pragma vectorize enable.
LV: Loop hints prevent vec

(4, 1, 1)

In [46]:
%timeit count_increasing_decreasing_avoid_conditionals(PREDICTABLE_DATA)
%timeit count_increasing_decreasing_avoid_conditionals(RANDOM_DATA)

725 µs ± 5.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
728 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
